# Word vectors

In [1]:
##imports
import numpy as np
import pandas as pd
import gensim
from bs4 import BeautifulSoup
import re
import nltk

D:\Softwares\Anaconda3\envs\tf2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
##getting data from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/kernels?sortBy=voteCount&group=everyone&pageSize=20&datasetId=134715
data_imdb = pd.read_csv(r'IMDB Dataset.csv')

In [3]:
#info
data_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
def clean_text(text):
    '''Cleans the text data'''
    ##remove html tags
    text = BeautifulSoup(text, "html.parser").get_text()
    return text.lower()

In [5]:
data_imdb['review'] = data_imdb['review'].apply(clean_text)

In [6]:
data_imdb.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


We will get the reviews text and train the word2vec using gensim and tensorflow. 

## Word2Vec using Gensim

In [7]:
##getting sentence wise data
list_sents = [nltk.word_tokenize(sent) for sent_tok in data_imdb.review for sent in nltk.sent_tokenize(sent_tok)]

In [0]:
##import gensim
from gensim.models import Word2Vec
##word2vec model ##this may take some time to execute. 
word2vec_model = Word2Vec(list_sents,##list of sentences, if you don;t have all the data in RAM, you can give file name to corpus_file 
                          size=50, ##output size of word emebedding 
                          window=4, ##window size
                          min_count=1, ## ignors all the words with total frquency lower than this
                          workers=5, ##number of workers to use
                          sg=1, ## skip gram
                          hs=0, ## 1 --> hierarchical, 0 --> Negative sampling
                          negative=5, ##How many negative samples
                          alpha=0.03, ##The initial learning rate
                          min_alpha=0.0001, ##Learning rate will linearly drop to min_alpha as training progresses.
                          seed = 54, ##random seed
                          iter=10,
                         compute_loss=True)##number of iterations

In [0]:
##getting a word vector
word2vec_model.wv['movie']

array([ 0.8166441 , -0.9765926 ,  0.6889766 , -0.15333726,  0.32603604,
        0.34232008,  0.1982183 ,  0.45404515, -0.27898434,  0.37556762,
        0.32830954,  0.4677273 , -0.15545918,  0.37998974,  0.4832581 ,
        0.5520326 ,  0.01102996, -0.73172736,  0.32559022,  0.0229142 ,
       -0.11138992,  0.13175151,  0.1802587 ,  0.10442656,  0.7089927 ,
        0.36721262,  0.2659526 ,  0.01060779,  0.09674501, -0.07720459,
       -0.1431406 , -0.04784476, -0.19753593, -0.01077123,  0.20721771,
       -0.308895  ,  0.3016016 ,  0.27222142, -0.04451592,  0.10112086,
        0.1615566 , -0.2219478 ,  0.14381573,  0.41217476,  0.08204534,
       -0.30486378, -0.279101  , -0.06951407,  1.008649  ,  0.46495098],
      dtype=float32)

In [0]:
word2vec_model.wv.most_similar(positive='movie')

[('film', 0.9484750628471375),
 ('film.it', 0.8743568658828735),
 ('film.but', 0.8640142679214478),
 ('movie.it', 0.862943708896637),
 ('movie.this', 0.8548593521118164),
 ('thing.i', 0.8546229004859924),
 ('one.it', 0.8504076600074768),
 ('film.what', 0.8469654321670532),
 ('train-wreck', 0.845355212688446),
 ('it.overall', 0.8342217206954956)]

In [0]:
word2vec_model.wv.most_similar(negative='movie')

[('-ascension', 0.3580598533153534),
 ('4.5*/5.0*', 0.2924154996871948),
 ('aghhh', 0.27601221203804016),
 ('x100', 0.2736707031726837),
 ('yikes.7/10', 0.2691265344619751),
 ('gwr', 0.2680243253707886),
 ('stupid.6', 0.26558661460876465),
 ("'aimee", 0.25137749314308167),
 ('purrrrrrrrrrrrrrrr', 0.24971842765808105),
 ('parhat', 0.24267500638961792)]

In [0]:
##saving 
word2vec_model.save('w2vmodel_gensim_model/w2vmodel')